In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras import Model, Input, layers
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.models import  Sequential
import warnings
import glob
warnings.filterwarnings("ignore")


In [5]:
train = pd.read_csv('D:/Projects/IoT Domain Analyst J Comp/Codes/Intensity/archive/insat_3d_ds - Sheet.csv')

In [ ]:
path = glob.glob('D:/Projects/IoT Domain Analyst J Comp/Codes/Intensity/archive/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED/*.jpg')
plt.figure(figsize=(64, 64))
for i in range(28):
    cur_img = mpimg.imread(path[i])
    # ax = plt.Subplot(7, 7, i+1)
    plt.imshow(cur_img.astype('uint8'))
    plt.axis('off')
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.20)

In [44]:
# train_data = train_datagen.flow_from_dataframe(train, directory='D:/Projects/IoT Domain Analyst J Comp/Codes/Intensity/archive/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED', 
#                                                subset='training', x_col='img_name', y_col='label', target_size=(64, 64), batch_size=32, class_mode='raw', shuffle=True)

# valid_data = train_datagen.flow_from_dataframe(train, directory='D:/Projects/IoT Domain Analyst J Comp/Codes/Intensity/archive/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED', 
#                                                subset='validation', x_col='img_name', y_col='label', target_size=(64, 64), batch_size=32, class_mode='raw',shuffle=True)


Found 0 validated image filenames.
Found 0 validated image filenames.


In [46]:
train_data = train_datagen.flow_from_dataframe(train, directory='D:/Projects/IoT Domain Analyst J Comp/Codes/Intensity/archive/insat3d_raw_cyclone_ds/CYCLONE_DATASET_FINAL', 
                                               subset='training', x_col='img_name', y_col='label', target_size=(64, 64), batch_size=32, class_mode='raw', shuffle=True)

valid_data = train_datagen.flow_from_dataframe(train, directory='D:/Projects/IoT Domain Analyst J Comp/Codes/Intensity/archive/insat3d_raw_cyclone_ds/CYCLONE_DATASET_FINAL', 
                                               subset='validation', x_col='img_name', y_col='label', target_size=(64, 64), batch_size=32, class_mode='raw',shuffle=True)


Found 105 validated image filenames.
Found 26 validated image filenames.


In [47]:
for image_batch, labels_batch in train_data:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 64, 64, 3)
(32,)


In [49]:
# def feature_extractor(input_shape):
#     input_shape = (64, 64, 3)
    # input_layer = Input(shape=input_shape)
model = Sequential([
    # Input(shape=input_shape)
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
    #32 convolution filters used each of size 3x3
    #again
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),

    #64 convolution filters used each of size 3x3
    #choose the best features via pooling

    #randomly turn neurons on and off to improve convergence
    Dropout(0.25),
    #flatten since too many dimensions, we only want a classification output
    Flatten(),
    #fully connected to get all relevant data
    Dense(128, activation='relu'),
    #one more dropout for convergence' sake :) 
    Dropout(0.5),
    #output a softmax to squash the matrix into output probabilities
    Dense(1, activation='selu')
])

In [97]:
model.compile(optimizer='nadam', loss='mse', metrics=['mean_squared_error'])

In [98]:
model.fit(train_data,validation_data=valid_data, epochs=50)

Epoch 1/50
4/4 [==============================] - 2s 219ms/step - loss: 4854.9561 - mean_squared_error: 4854.9561 - val_loss: 1299.3737 - val_mean_squared_error: 1299.3737
Epoch 2/50
4/4 [==============================] - 1s 200ms/step - loss: 4854.9561 - mean_squared_error: 4854.9561 - val_loss: 1299.3735 - val_mean_squared_error: 1299.3735
Epoch 3/50
4/4 [==============================] - 1s 242ms/step - loss: 4854.9561 - mean_squared_error: 4854.9561 - val_loss: 1299.3737 - val_mean_squared_error: 1299.3737
Epoch 4/50
4/4 [==============================] - 1s 200ms/step - loss: 4854.9561 - mean_squared_error: 4854.9561 - val_loss: 1299.3735 - val_mean_squared_error: 1299.3735
Epoch 5/50
4/4 [==============================] - ETA: 0s - loss: 4854.9561 - mean_squared_error: 4854.9561

KeyboardInterrupt: 

In [55]:
model.save('mse.h5', overwrite=True)

In [53]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 62, 62, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 60, 60, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 58, 58, 32)        18464     
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 58, 58, 32)      

In [9]:
from keras.models import load_model
from keras.utils import load_img, img_to_array 
import numpy as np

# Load the trained model
model = load_model('mse.h5')

# Load and preprocess the new image
img = load_img('D:/Projects/IoT Domain Analyst J Comp/Codes/Intensity/archive/insat3d_raw_cyclone_ds/CYCLONE_DATASET_FINAL/25.jpg', target_size=(64, 64))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.0  # Rescale pixel values to [0, 1]

# Make the prediction and get the intensity value
intensity_pred = model.predict(x)[0][0]
print(intensity_pred)


1/1 [==============================] - 0s 51ms/step
46.693237
